In [1]:
# Import libraries
from functions import *

import json
import os
import subprocess
import time

# Creating a transaction with an absolute timelock (nLocktime)

In this section we'll cover how to create a transaction using absolute timelocks (nLocktime).

Absolute timelocks are used for protection against [fee sniping](https://bitcoinops.org/en/topics/fee-sniping/#:~:text=Fee%20sniping%20occurs%20when%20a,who%20originally%20created%20those%20blocks.), and in layer 2 protocols (in conjunction with OP_CHECKLOCKTIMEVERIFY) such lightning HTLC transactions, cross chain atomic swaps.

This section assumes knowledge of P2WPKH transactions.


### Reading
- https://medium.com/summa-technology/bitcoins-time-locks-27e0c362d7a1

### Scenario
Kim is an old grandmother who wants to pass some of her wealth (0.1 btc) on to her grandson, Peter. However, Peter is still a child and Kim doesn't trust Peter to spend his bitcoin wisely. Kim would rather wait until Peter turns 18 before he's able to spend his bitcoin. Kim doesn't know how much longer she'll be around for, and she doesn't trust anyone else to handle her money. To give Peter bitcoin in a way which will only allow him to spend it once he's 18, Kim will do this by giving Peter a signed transaction for 0.1 btc with an absolute timelock set to expire on his 18th birthday.

Note that 1 year corresponds to about 50,000 blocks. To save us having to mine tens of thousands of regtest blocks (which may take a few minutes), we'll instead suppose Peter's 18th birthday is happening much sooner, at block 500.

### Create a P2WPKH UTXO

For this example, Kim is spending from a P2WPKH output, although the same mechanism would work with any other type of output. We'll use the same code from the P2WPKH example to create the utxo.

In [2]:
sender_privkey = bytes.fromhex("1111111111111111111111111111111111111111111111111111111111111111")
sender_pubkey = privkey_to_pubkey(sender_privkey)
sender_p2wpkh_addr = pk_to_p2wpkh(sender_pubkey, network = "regtest")
print("sender's p2wpkh address: " + sender_p2wpkh_addr)

sender's p2wpkh address: bcrt1ql3e9pgs3mmwuwrh95fecme0s0qtn2880hlwwpw


#### Create Kim's utxo with 0.101 btc (0.001 btc is for the next tx fee)

Now that we have the address, we'll fund it using the python code we used in the previous sections, `create_regtest_utxo.py`.

In [3]:
txid_to_spend, index_to_spend = create_regtest_utxo(sender_p2wpkh_addr, 0.101)
print(f"txid: {txid_to_spend}, {index_to_spend}")

txid: 6836ae2a40f84962f7983cde782e78a50e667a7a36dc003efa3dad86dd856811, 0


## Creating a timelocked transaction

Now that we have created Kim's P2WPKH utxo, we can create her timelocked transaction spending from it. Let's say Peter's address is `bcrt1ql3e9pgs3mmwuwrh95fecme0s0qtn2880hlwwpw`, which corresponds to the scriptPubkey `0014fc7250a211deddc70ee5a2738de5f07817351cef`. We'll also set the amount to spend the entire utxo, excluding the miner fee (of 0.001 btc).

In [4]:
output1_spk = bytes.fromhex("0014fc7250a211deddc70ee5a2738de5f07817351cef")
output1_value_sat = int(float("0.1") * 100000000)

### nLocktime

Times are expressed as an unsigned 32 bit (4 byte) integer. If time_lock is 0, it's ignored. If it is 500,000,000 or above, it's treated as a unix timestamp. To set nLocktime, we'll use the built in python method `to_bytes` to convert the integer value for the blockheight into an unsigned 32 bit integer.

In [5]:
# let's suppose Peter's 18th birthday is estimated to happen at block 500
locktime_int = 500
locktime = locktime_int.to_bytes(4, byteorder="little", signed=False)
print("nLocktime: ", locktime.hex())

nLocktime:  f4010000


Note that `nLocktime` will be ignored if `nSequence` is set to the largest value (`0xffffffff`). Using `nSequence` to signal in this way is an artifact from Satoshi's half-baked time-lock implementation and at this point it would require a hardfork to change it. To enable `nLocktime` we'll set `nSequence` to `0xffff fffe`. Note that this value for `nSequence` is also the default set by bitcoin core.

In [6]:
sequence = bytes.fromhex("ffff fffe")
print("sequence: ", sequence.hex())

sequence:  fffffffe


### Create an unsigned transaction

Using the values for `nLocktime` that we set above `nSequence`, we'll create the unsigned transaction and sign it in the same way we did when creating a regular P2WPKH transaction.

In [7]:
# VERSION
# version '2' indicates that we may use relative timelocks (BIP68)
version = bytes.fromhex("0200 0000")

# MARKER (new to segwit)
marker = bytes.fromhex("00")

# FLAG (new to segwit)
flag = bytes.fromhex("01")

# INPUTS
# We have just 1 input
input_count = bytes.fromhex("01")

# Convert txid and index to bytes (little endian)
txid = (bytes.fromhex(txid_to_spend))[::-1]
index = index_to_spend.to_bytes(4, byteorder="little", signed=False)

# For the unsigned transaction we use an empty scriptSig
scriptsig = bytes.fromhex("")

# SEQUENCE
# set above

inputs = (
    txid
    + index
    + varint_len(scriptsig)
    + scriptsig
    + sequence
)

# OUTPUTS
# 0x02 for out two outputs
output_count = bytes.fromhex("01")

# OUTPUT 1 
output1_value = output1_value_sat.to_bytes(8, byteorder="little", signed=True)
# 'output1_spk' already defined at the start of the script

outputs = (
    output1_value
    + varint_len(output1_spk)
    + output1_spk
)

# LOCKTIME
# set above

unsigned_tx = (
    version
    + input_count
    + inputs
    + output_count
    + outputs
    + locktime
)
print("unsigned_tx: ", unsigned_tx.hex())

unsigned_tx:  0200000001116885dd86ad3dfa3e00dc367a7a660ea5782e78de3c98f76249f8402aae36680000000000fffffffe018096980000000000160014fc7250a211deddc70ee5a2738de5f07817351ceff4010000


### Signing the transaction

Segwit transaction signing scheme described in [BIP143](https://github.com/bitcoin/bips/blob/master/bip-0143.mediawiki)


In [8]:
pk_hash = hash160(sender_pubkey)
scriptcode = bytes.fromhex("76a914" + pk_hash.hex() + "88ac")

input_amount_sat = int(0.101 * 100_000_000)
value = input_amount_sat.to_bytes(8, byteorder="little", signed=False)

hashPrevOuts = hash256(txid + index)
hashSequence = hash256(sequence)
hashOutputs = hash256(outputs)
sighash_type = bytes.fromhex("0100 0000") # SIGHASH_ALL

tx_digest_preimage = (
    version
    + hashPrevOuts
    + hashSequence
    + txid
    + index
    + varint_len(scriptcode)
    + scriptcode
    + value
    + sequence
    + hashOutputs
    + locktime
    + sighash_type
)
print(tx_digest_preimage.hex())

0200000035abbc8135db0744e6bd6336e0f318b6c5bd2cfe88db76dbaba31dd047f8956bbbdebfac1cd12080fb29f8919f7f8be7bdb21a89c8d09e8a59fc7bb1d2a73796116885dd86ad3dfa3e00dc367a7a660ea5782e78de3c98f76249f8402aae3668000000001976a914fc7250a211deddc70ee5a2738de5f07817351cef88ac201d9a0000000000fffffffe62580c97423350cc6d13c95a8203b8dd1962f9f34ecaa00a7542aa14f6e2b1a3f401000001000000


Hash this transaction and produce an ecdsa signature on it. 

In [9]:
# Create sigHash to be signed
sighash = hash256(tx_digest_preimage)

# Sign the sigHash with the input private key
signing_key = ecdsa.SigningKey.from_string(sender_privkey, curve=ecdsa.SECP256k1) 
signature = signing_key.sign_digest(sighash, sigencode=ecdsa.util.sigencode_der_canonize)

# Append SIGHASH_ALL to the signature
signature = signature + bytes.fromhex("01")

# Witness field
witness = (
    # indicate the number of stack items for the txin
    # 2 items for signature and pubkey
    bytes.fromhex("02")
    + pushbytes(signature)
    + pushbytes(sender_pubkey)
)

# tx_in with our new sigScript containing the signature we just created
inputs_signed = (
    txid
    + index
    + varint_len(scriptsig)
    + scriptsig
    + sequence
)

# the final signed transaction
signed_tx = (
    version
    + marker
    + flag
    + input_count
    + inputs_signed
    + output_count
    + outputs
    + witness
    + locktime
)

print("signed transaction: ",signed_tx.hex())

signed transaction:  02000000000101116885dd86ad3dfa3e00dc367a7a660ea5782e78de3c98f76249f8402aae36680000000000fffffffe018096980000000000160014fc7250a211deddc70ee5a2738de5f07817351cef024830450221008675a5136403cf2480be964a0181d3164735047c6026830b52052a12f2969b7002206ca364877eea0e44aead79f75e2826ae0d617bf2d25763cfbf0fd975cdb8d4170121034f355bdcb7cc0af728ef3cceb9615d90684bb5b2ca5f859ab0f0b704075871aaf4010000


### Scenario
Kim has now successfully created a transaction which can only be spent by Peter after his 18th birthday (block 500)! Note that Peter won't be able to broadcast this transaction until the block height has been reached.

If he were to try broadcasting the transaction, it would result in the following `non-final` error.

In [10]:
height = subprocess.getoutput("bitcoin-cli -regtest getblockcount ")
print(height)

invalid_attempt = subprocess.getoutput(f"bitcoin-cli -regtest sendrawtransaction {signed_tx.hex()}")
print(invalid_attempt)

102
error code: -26
error message:
non-final


Now we'll simulate 400 blocks passing (to reach block 502), and see what happens if Peter tries to broadcast his transaction again.

In [11]:
subprocess.getoutput(f"bitcoin-cli -regtest -generate 400")
height = subprocess.getoutput("bitcoin-cli -regtest getblockcount ")
print("blockheight: ", height)

valid_spend = subprocess.getoutput(f"bitcoin-cli -regtest sendrawtransaction {signed_tx.hex()}")
print(valid_spend)

blockheight:  502
234532d29359fca8a62aa21e31bd9bf4be63cb74ad62dcdd984d36362bccb7b8


This time the transaction is successfully broadcasted and Peter has full ownership of his gift. 

To view the decoded transaction, uncomment the lines below.

In [12]:
# decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + signed_tx.hex())
# print(decoded)

In [13]:
# stop bitcoin core
subprocess.getoutput("bitcoin-cli -regtest stop")

'Bitcoin Core stopping'

## Quiz

- 1. What would happen if Kim created the transaction with `nSequence` set to `0xffffffff`?
- 2. Is Peter guaranteed to be able to broadcast his signed transaction after block 500?

## Answers
- 1. nLocktime would not have been enforced, meaning Peter would have been able to broadcast the transaction immediately.
- 2. No. Kim may create a new transaction that spends the input before Peter has a chance to broadcast his transaction.

## Exercise
- Try creating a timelock using the unix timestamp instead of the blockheight. Note that the timestamp in regtest blocks is taken from the system's clock.